In [2]:
import pandas as pd 
import numpy as np
from pathlib import Path
import glob 
import importlib
from functions import open_workout_file, get_all_workout_files
from functions import calculate_weighted_set_volume, extract_bodyweight_volume, extract_volume, get_workout_name, get_workout_date
import functions

In [3]:
# import workout files 
workout_files = get_all_workout_files()
print(f"Found {len(workout_files)} workout files")

Found 11 workout files


In [4]:
line = open_workout_file(workout_files[0])
print("First line of first workout file:", line[0])
print("Fifth line of first workout file:", line[5])

First line of first workout file: Sep 26 Workout

Fifth line of first workout file: Set 2 : 2 kg  x 11



In [5]:
calculate_weighted_set_volume(line[5])

22.0

In [6]:
total = extract_volume(workout_files[0], bodyweight=80)
print(f"Total volume: {total} kg")

Total volume: 224.0 kg


In [7]:
for file in workout_files:
    total = extract_volume(file, bodyweight=80)
    print(f"{file}: {total} kg")

../data/raw/workouts/26_09_25.txt: 224.0 kg
../data/raw/workouts/09_10_25.txt: 2204.0 kg
../data/raw/workouts/11_10_25.txt: 7514.0 kg
../data/raw/workouts/05_10_25.txt: 5338.0 kg
../data/raw/workouts/06_10_25.txt: 9711.0 kg
../data/raw/workouts/24_09_25.txt: 5653.0 kg
../data/raw/workouts/12_10_25.txt: 6117.0 kg
../data/raw/workouts/15_10_25.txt: 11275.0 kg
../data/raw/workouts/07_10_25.txt: 4451.0 kg
../data/raw/workouts/25_09_25.txt: 5469.5 kg
../data/raw/workouts/28_09_25.txt: 4935.0 kg


In [8]:
for file in workout_files:
    print(f"Processing: {file}")
    total = extract_volume(file, bodyweight=80)
    print(f"  Total: {total} kg")

Processing: ../data/raw/workouts/26_09_25.txt
  Total: 224.0 kg
Processing: ../data/raw/workouts/09_10_25.txt
  Total: 2204.0 kg
Processing: ../data/raw/workouts/11_10_25.txt
  Total: 7514.0 kg
Processing: ../data/raw/workouts/05_10_25.txt
  Total: 5338.0 kg
Processing: ../data/raw/workouts/06_10_25.txt
  Total: 9711.0 kg
Processing: ../data/raw/workouts/24_09_25.txt
  Total: 5653.0 kg
Processing: ../data/raw/workouts/12_10_25.txt
  Total: 6117.0 kg
Processing: ../data/raw/workouts/15_10_25.txt
  Total: 11275.0 kg
Processing: ../data/raw/workouts/07_10_25.txt
  Total: 4451.0 kg
Processing: ../data/raw/workouts/25_09_25.txt
  Total: 5469.5 kg
Processing: ../data/raw/workouts/28_09_25.txt
  Total: 4935.0 kg


In [9]:
name = get_workout_name(workout_files[0])
print(name)

Sep 26 Workout


In [10]:
date = get_workout_date(workout_files[0])
print(date)

2025-09-26


### Create dataframe for workout data

In [11]:
# Create lists to store data
dates = []
names = []
volumes = []

# Loop through all workout files
for file in workout_files:
    date = get_workout_date(file)
    name = get_workout_name(file)
    volume = extract_volume(file, bodyweight=80)
    
    dates.append(date)
    names.append(name)
    volumes.append(volume)

# Create DataFrame
workout_df = pd.DataFrame({
    'date': dates,
    'workout_name': names,
    'total_volume': volumes
})

print(workout_df)

          date    workout_name  total_volume
0   2025-09-26  Sep 26 Workout         224.0
1   2025-10-09   Oct 9 Workout        2204.0
2   2025-10-11    BACK & DELTS        7514.0
3   2025-10-05      Upper MP B        5338.0
4   2025-10-06   Oct 6 Workout        9711.0
5   2025-09-24      Upper MP A        5653.0
6   2025-10-12      Upper MP B        6117.0
7   2025-10-15  Oct 15 Workout       11275.0
8   2025-10-07      Upper MP A        4451.0
9   2025-09-25    BACK & DELTS        5469.5
10  2025-09-28      Upper MP B        4935.0


In [13]:
# save to csv
workout_df.to_csv('../data/processed/workout_summary.csv', index=False)
print("Workout summary saved to ../data/processed/workout_summary.csv")

Workout summary saved to ../data/processed/workout_summary.csv
